In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets
from torchvision.transforms import ToTensor

In [ ]:
from model import Net
# from train import train

In [ ]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
BATCH_SIZE = 64
EPOCHS = 10

In [ ]:
training_data = datasets.CIFAR10(root="data", train=True, download=True, transform=ToTensor())
train_loader = torch.utils.data.DataLoader(training_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=8)
# TODO: divide into train and valid

test_data = datasets.CIFAR10(root="data", train=False, download=True, transform=ToTensor())
test_loader = torch.utils.data.DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=8)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [ ]:
labels_map = {
    0: "Airplane",
    1: "Automobile",
    2: "Bird",
    3: "Cat",
    4: "Deer",
    5: "Dog",
    6: "Frog",
    7: "Horse",
    8: "Ship",
    9: "Truck",
}
figure = plt.figure(figsize=(8, 8))
cols, rows = 3, 3
for i in range(1, cols * rows + 1):
    sample_idx = torch.randint(len(training_data), size=(1,)).item()
    img, label = training_data[sample_idx]
    figure.add_subplot(rows, cols, i)
    plt.title(labels_map[label])
    plt.axis("off")
    plt.imshow(img.permute(1, 2, 0))
plt.show()

In [ ]:
training_data.data.shape

In [ ]:
train_loader.dataset.data.shape

In [ ]:
model = Net().to(DEVICE)

In [ ]:
# optimizer = optim.Adam(model.parameters(), lr=0.001)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum = 0.9)
criterion = nn.CrossEntropyLoss()
total_step = len(train_loader)

Allora, l'idea e' quella di implementare la contrastive loss. Cio' significa ridurre la distanza tra vettori simili e aumentare la distanza tra vettori diversi $\rightarrow$ loss in funzione della distanza tra i due vettori: se appartenenti alla stessa classe e bassa distanza, loss bassa, e viceversa

In [ ]:
class ContrastiveLoss(nn.Module):
    def __init__(self, margin=1.0):
        super().__init__()
        self.margin = margin

    def forward(self, z1, z2, y):
        """
        z1, z2: embeddings of shape (B, D)
        y: binary labels (1 = same class, 0 = different class)
        """

        # Euclidean distance
        dist = torch.norm(z1 - z2, p=2, dim=1)

        loss_pos = y * dist.pow(2)
        loss_neg = (1 - y) * torch.clamp(self.margin - dist, min=0).pow(2)

        loss = loss_pos + loss_neg
        return loss.mean()

In [ ]:
class ContrastiveLoss(nn.Module):
    def __init__(self):
        super(ContrastiveLoss, self).__init__()

    def forward(self, output, target):
        criterion = nn.CrossEntropyLoss()
        loss1 = criterion(output, target)
        loss2 = 0
        return loss1 + loss2

In [ ]:
for epoch in range(EPOCHS):
    for i, (images, labels) in enumerate(train_loader):
        # Move tensors to the configured device
        images = images.to(DEVICE)
        labels = labels.to(DEVICE)
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        loss.backward()
        optimizer.step()

    print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, EPOCHS, i+1, total_step, loss.item()))

In [ ]:
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(DEVICE)
        labels = labels.to(DEVICE)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        del images, labels, outputs

    print('Accuracy of the network on the {} test images: {} %'.format(10000, 100 * correct / total))